In [3]:
from transformers import pipeline
import pandas as pd
import re

arr = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]


def fun(s):
    s = s.lower().split()
    ans = ""
    for i in range(len(s)):
        for j in range(10):
            if(s[i].count(arr[j])>0):
                ans += str(j)
                break
        if(len(ans)==3):
            break
    return ans


file_path = './nlp.jsonl'
data = pd.read_json(path_or_buf=file_path, lines=True,dtype={'key':'str', 'transcript':'str', 'tool':'str', 'heading':'str', 'targtet':'str'})

tool_answerer = pipeline("question-answering", model="toolmodel")
target_answerer = pipeline("question-answering", model="targetmodel")
heading_answerer = pipeline("question-answering", model="headingmodel")

for i in range((len(data))):
    
    context = data[i:i+1]["transcript"].item()
    
    question = "What is the tool to use to destroy target"
    tool = tool_answerer(question=question, context=context)["answer"]
    if(tool != "EMP"):
        tool = tool.lower()
    if(tool == "machine guns"):
        tool = "machine gun"
        
    
    question = "what is the target to destroy"
    target = target_answerer(question=question, context=context)["answer"].lower()
    if(target[-1] == 's'):
        target = target[0:-1]
        
    
    
    question = "What is the heading"
    heading = heading_answerer(question=question, context=context)["answer"]
    heading = fun(heading)
    
    answer = {"tool":tool, "target":target, "heading":heading}
    actual_answer = {"tool":data[i:i+1]["tool"].item(), "target":data[i:i+1]["target"].item(), "heading":data[i:i+1]["heading"].item()}
    
    if(answer != actual_answer):
        print(i, answer, actual_answer, context)
    
    